In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
%cd drive/MyDrive/Colab\ Notebooks/M2_DL/

/content/drive/MyDrive/Colab Notebooks/M2_DL


# Sinwave

In [5]:
import numpy as np

# lecture des donnees et découpage en sequence de seq_len elements
def load_data(filename, seq_len):
  f = open(filename, 'rb').read()
  data = np.array(f.decode().split('\n'), dtype = np.float32)
  sequence_length = seq_len + 1
  result = []
  for index in range(len(data) - sequence_length):
    result.append(data[index: index + sequence_length])
  result = np.array(result)
  np.random.shuffle(result)
  row = round(0.9 * result.shape[0])
  train = result[:int(row), :] # 90 % des exemples pour l’apprentissage
  x_train = train[:, :-1] # On prend les séquences jusqu’à l’avant dernier élément
  y_train = train[:, -1] # On prend le dernier élément comme sortie à apprendre
  x_test = result[int(row):, :-1]
  y_test = result[int(row):, -1]
  # on transforme en un tenseur de dimension 3 avec une seule feature
  x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
  x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
  return x_train, y_train, x_test, y_test

In [7]:
import tensorflow as tf
from tensorflow.keras import layers, Sequential

In [8]:
model = Sequential()
model.add(layers.LSTM(50, input_shape=(50,1), return_sequences=True))
model.add(layers.Dropout(0.2))
model.add(layers.LSTM(100, return_sequences=False))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(1))
model.compile(loss="mse", optimizer="rmsprop", metrics = ["mae"])

In [9]:
X_train, y_train, X_test, y_test = load_data('sinwave.csv', 50)
model.fit(X_train, y_train, batch_size=512, epochs=1, validation_split=0.05)
predict = model.predict (X_test)

16/16 [==============================] - 1s 21ms/step


#S&P 500

In [10]:
def normalise_windows(window_data):
  normalised_data = []
  for window in window_data:
    normalised_window = [((float(p) / float(window[0])) - 1) for p in window]
  normalised_data.append(normalised_window)
  return normalised_data

In [18]:
sum(1 for i in f.decode().split("\n") if i == '')

1

In [12]:
seq_len = 50
f = open("sp500.csv", "rb").read()
data = np.array(f.decode().split('\n'), dtype = np.float32)
sequence_length = seq_len + 1
result = []
for index in range(len(data) - sequence_length):
  result.append(data[index: index + sequence_length])
result = np.array(result)
np.random.shuffle(result)
row = round(0.9 * result.shape[0])
train = result[:int(row), :] # 90 % des exemples pour l’apprentissage
x_train = train[:, :-1] # On prend les séquences jusqu’à l’avant dernier élément
y_train = train[:, -1] # On prend le dernier élément comme sortie à apprendre
x_test = result[int(row):, :-1]
y_test = result[int(row):, -1]
# on transforme en un tenseur de dimension 3 avec une seule feature
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

ValueError: ignored

In [11]:
load_data("sp500.csv", 50)

ValueError: ignored

# Text Generation

In [21]:
import keras

path = keras.utils.get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('Corpus length:', len(text))

600901/600901 [==============================] - 0s 1us/step
Corpus length: 600893


In [26]:
maxlen = 60
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
  sentences.append(text[i: i + maxlen])
  next_chars.append(text[i + maxlen])
print('Number of sequences:', len(sentences))
chars = sorted(list(set(text)))
print('Unique characters:', len(chars))
char_indices = dict((char, chars.index(char)) for char in chars)
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=bool)
y = np.zeros((len(sentences), len(chars)), dtype=bool)
for i, sentence in enumerate(sentences):
  for t, char in enumerate(sentence):
    x[i, t, char_indices[char]] = 1
  y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 200278
Unique characters: 57
Vectorization...


In [27]:
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

In [29]:
optimizer = keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [30]:
def sample(preds, temperature=1.0):
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds) / temperature
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  probas = np.random.multinomial(1, preds, 1)
  return np.argmax(probas)

In [ ]:
import random
import sys

for epoch in range(1, 60):
  model.fit(x, y, batch_size=128, epochs=1)
  print('epoch', epoch)
  start_index = random.randint(0, len(text) - maxlen - 1)
  generated_text = text[start_index: start_index + maxlen]
  for temperature in [0.2, 0.5, 1.0, 1.2]:
    sys.stdout.write(generated_text)
    print('\n\n------ temperature:', temperature)
    for i in range(400):
      sampled = np.zeros((1, maxlen, len(chars)))
      for t, char in enumerate(generated_text):
        sampled[0, t, char_indices[char]] = 1.
      preds = model.predict(sampled, verbose=0)[0]
      next_index = sample(preds, temperature)
      next_char = chars[next_index]
      generated_text += next_char
      generated_text = generated_text[1:]
      sys.stdout.write(next_char)

1565/1565 [==============================] - 281s 177ms/step - loss: 1.9754
epoch 1
 is no eternal justice.


54

=falsehood.=--why do men, as a

------ temperature: 0.2
 can the conscience the conscience and the conscience of the string the sense of the string the stinct the morality the man he conscience of the can the conscience of the conscience of the conscience and the conscience and something the conscience of the stance of the stinct the conscience of the stinction of the string the ston the string the conscience and the some the conscience and the stinctig the conscience and the some the conscience and the stincti

------ temperature: 0.5
on and southing onesen the consequence the please and some the cree much the fastions the child in the more the hand not it is ideas with pution of the proplans in the ston and stonds might in the compares even all and sour the stinct the fartity and and conscience to the resting of the conscience of unders and and for and conscience the sens